# DAT210x - Programming with Python for DS

## Module6- Lab3

This code is intentionally missing! Read the directions on the course lab page!

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split

C:\Users\alloq\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Load up the /Module6/Datasets/parkinsons.data data set into a variable X, being sure to drop the name column.

In [2]:
X = pd.read_csv('Datasets/parkinsons.data')
X.drop('name', axis = 1, inplace = True)
X.head()
X.describe()
X.isnull().sum() # No NaNs!
X.dtypes # All object types are correct!
X.isnull().sum()

MDVP:Fo(Hz)         0
MDVP:Fhi(Hz)        0
MDVP:Flo(Hz)        0
MDVP:Jitter(%)      0
MDVP:Jitter(Abs)    0
MDVP:RAP            0
MDVP:PPQ            0
Jitter:DDP          0
MDVP:Shimmer        0
MDVP:Shimmer(dB)    0
Shimmer:APQ3        0
Shimmer:APQ5        0
MDVP:APQ            0
Shimmer:DDA         0
NHR                 0
HNR                 0
status              0
RPDE                0
DFA                 0
spread1             0
spread2             0
D2                  0
PPE                 0
dtype: int64

Splice out the status column into a variable y and delete it from X.

In [3]:
y = X.status
X.drop('status', axis = 1, inplace = True)
X.columns # 'status' has been dropped from X

Index(['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1',
       'spread2', 'D2', 'PPE'],
      dtype='object')

Wait a second. Pull open the dataset's label file from: https://archive.ics.uci.edu/ml/datasets/Parkinsons
Look at the units on those columns: Hz, %, Abs, dB, etc. What happened to transforming your data? With all of those units
interacting with one another, some pre-processing is surely in order. Right after you splice out the status column, but before
you process the train/test split, inject SciKit-Learn pre-processing code. Unless you have a good idea which one is going to work
best, you're going to have to try the various pre-processors one at a time, checking to see if they improve your predictive accuracy.
Experiment with Normalizer(), MaxAbsScaler(), MinMaxScaler(), and StandardScaler().

In [4]:
T = preprocessing.StandardScaler().fit_transform(X)
#T = preprocessing.MinMaxScaler().fit_transform(X)
#T = preprocessing.Normalizer().fit_transform(X)
#T = preprocessing.MaxAbsScaler().fit_transform(X)
#T = X # No Change

The accuracy score keeps creeping upwards. Let's have one more go at it. Remember how in a previous lab we discovered that SVM's
are a bit sensitive to outliers and that just throwing all of our unfiltered, dirty or noisy data at it, particularly in
high-dimensionality space, can actually cause the accuracy score to suffer?
Well, let's try to get rid of some useless features. Immediately after you do the pre-processing, run PCA on your dataset. The
original dataset has 22 columns and 1 label column. So try experimenting with PCA n_component values between 4 and 14. Are you able
to get a better accuracy?

No, the accuracy levels off at the same value as before from 7 components onwards.

If you are not, then forget about PCA entirely, unless you want to visualize your data. However if you are able to get a higher score,
then be *sure* keep that figure in mind, and comment out all the PCA code.
In the same spot, run Isomap on the data, before sending it to the train / test split. Manually experiment with every inclusive
combination of n_neighbors between 2 and 5, and n_components between 4 and 6. Are you able to get a better accuracy?

Isomap with neighbors = 3, components = 5 or 6 provide the highest score.

Program a naive, best-parameter searcher by creating a nested for-loops. The outer for-loop should iterate a variable C
from 0.05 to 2, using 0.05 unit increments. The inner for-loop should increment a variable gamma from 0.001 to 0.1, using
0.001 unit increments. As you know, Python ranges won't allow for float intervals, so you'll have to do some research on
NumPy ARanges, if you don't already know how to use them.

Since the goal is to find the parameters that result in the model having the best score, you'll need a best_score = 0 variable
that you initialize outside of the for-loops. Inside the for-loop, create a model and pass in the C and gamma parameters into
the class constructor. Train and score the model appropriately. If the current best_score is less than the model's score, then
update the best_score, being sure to print it out, along with the C and gamma values that resulted in it.   

In [5]:
best_score = 0
     
for k in range(2, 6):
    for l in range(4, 7):
        iso = Isomap(n_neighbors = k, n_components = l)
        X_iso = iso.fit_transform(T)
        X_train, X_test, y_train, y_test = train_test_split(X_iso, y, test_size = 0.3, random_state = 7)
        for i in np.arange(start = 0.05, stop = 2.05, step = 0.05):
            for j in np.arange(start = 0.001, stop = 0.101, step = 0.001):
                model = SVC(C = i, gamma = j)
                model.fit(X_train, y_train)
                score = model.score(X_test, y_test)
                if score > best_score:
                    best_score = score
                    best_C = model.C
                    best_gamma = model.gamma
                    best_n_neighbors = iso.n_neighbors
                    best_n_components = iso.n_components
print("The highest score obtained:", best_score)
print("C value:", best_C )
print("gamma value:", best_gamma)
print('isomap n_neighbors', best_n_neighbors)
print('isomap n_components', best_n_components)
                        


The highest score obtained: 0.9661016949152542
C value: 1.7000000000000002
gamma value: 0.060000000000000005
isomap n_neighbors 2
isomap n_components 4


In [6]:
best_score = 0
for i in range(4, 15):
    pca = PCA(n_components = i)
    X_pca = pca.fit_transform(T)
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size = 0.3, random_state = 7)
    for i in np.arange(start = 0.05, stop = 2.05, step = 0.05):
            for j in np.arange(start = 0.001, stop = 0.101, step = 0.001):
                model = SVC(C = i, gamma = j)
                model.fit(X_train, y_train)
                score = model.score(X_test, y_test)
                if score > best_score:
                    best_score = score
                    best_C = model.C
                    best_gamma = model.gamma
                    best_n_components = pca.n_components
            
print("The highest score obtained:", best_score)
print("C value:", best_C )
print("gamma value:", best_gamma)
print('pca n_components', best_n_components)

The highest score obtained: 0.9322033898305084
C value: 1.7500000000000002
gamma value: 0.099
pca n_components 7


If you are not, then forget about isomap entirely, unless you want to visualize your data. However if you are able to get a higher
score, then be *sure* keep that figure in mind.
If either PCA or Isomap helped you out, then uncomment out the appropriate transformation code so that you have the highest accuracy
possible.

Lab Question 1:
What accuracy did you score?
Answer: 0.847457627118644

Question 2: After running your assignment again, what is the highest accuracy score you are able to get?
Answer: 0.8983050847457628

Question 3: After trying all of these scalers, what is the new highest accuracy score you're able to achieve?
Answer: 0.9322033898305084

Question 4: What is your highest accuracy score on this assignment to date?
Answer: The highest score obtained: 0.9661016949152542
C value: 1.7000000000000002
gamma value: 0.060000000000000005
isomap n_neighbors 2
isomap n_components 4